In [1]:
import inspect
import time

import numpy as np
from IPython.display import Markdown as md

from src.equation import init_equation
from src.methods import simple_iteration, gradient_descent

## 2) Реализовать итерационный метод градиентного спуска

In [2]:
md(f"""
### Реализованный метод градиентного спуска (см. [src/methods.py](src/methods.py))):

```python
{inspect.getsource(gradient_descent)}
```
""")


### Реализованный метод градиентного спуска (см. [src/methods.py](src/methods.py))):

```python
def gradient_descent(A: np.matrix, B: np.ndarray, eps: float = 1e-6) -> Coroutine[np.ndarray, None, None]:
    N = A.shape[0]
    u = np.random.random(N)
    u_old = np.zeros(N)
    while np.linalg.norm(u - u_old) / np.linalg.norm(B) >= eps:
        u_old = u
        r = A @ u - B
        r = np.squeeze(np.asarray(r))
        A_star_r = np.squeeze(np.asarray(np.conj(A.T) @ r))
        u = u - np.linalg.norm(A_star_r) ** 2 / np.linalg.norm(A @ A_star_r) ** 2 * A_star_r
        u = np.squeeze(np.asarray(u))
        yield u

```


In [3]:
equation = init_equation(300)
A_, B_ = equation

In [4]:
np.linalg.norm(list(gradient_descent(A_, B_))[-1] - np.linalg.solve(A_, B_))

8.55445763455629e-08

## 3) При одинаковых значениях ε сравнить эффективность и качество сходимости итерационных методов: метода простой итерации и градиентного спуска.

In [5]:
np.random.seed(1000 - 7)

In [6]:
start = time.time()
simple_iteration_history = list(simple_iteration(A_, B_))
simple_iteration_time = time.time() - start

start = time.time()
gradient_descent_history = list(gradient_descent(A_, B_))
gradient_descent_time = time.time() - start

In [7]:
import plotly.graph_objects as go


go.Figure(
    data=[
        go.Scatter(
            x=np.array(range(len(simple_iteration_history))) + 1,
            y=[np.linalg.norm(u - np.linalg.solve(A_, B_)) for u in simple_iteration_history],
            name="Метод простой итерации",
        ),
        go.Scatter(
            x=np.array(range(len(gradient_descent_history))) + 1,
            y=[np.linalg.norm(u - np.linalg.solve(A_, B_)) for u in gradient_descent_history],
            name="Метод градиентного спуска",
        ),
    ],
    layout=go.Layout(
        title="Зависимость нормы разности решения от нормы разности решения, полученного с помощью numpy.linalg.solve",
        xaxis_title="Номер итерации",
        yaxis_title="Норма разности решения",
    ),
).show()

In [8]:
go.Figure(
    data=[
        go.Bar(
            x=["Метод простой итерации", "Метод градиентного спуска"],
            y=[simple_iteration_time, gradient_descent_time],
            name="Время работы",
            marker=dict(
                color=["#1f77b4", "#ff7f0e"],
            ),
        ),
    ],
    layout=go.Layout(
        title="Время работы методов",
        xaxis_title="Метод",
        yaxis_title="Время работы, с",
    ),
).show()